In [195]:
# install csv-to-sqlite to transform csv to sqlite
#!pip install csv-to-sqlite

# import csv_to_sqlite 

# # read and write csv to sqlite
# options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
# input_files = ["orders.csv", "order_items.csv"] # pass in a list of CSV files
# csv_to_sqlite.write_csv(input_files, "output.sqlite", options)
import pandas as pd

In [196]:

# SQLAlchemy
from sqlalchemy import create_engine

# Path to sqlite
database_path = "output.sqlite"

In [197]:
# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()


In [200]:
# Query All Records in the the Database
order_df = pd.read_sql("SELECT * FROM orders", conn)
orders_df = order_df.rename(columns={'delivered_to':'order_id'})

In [201]:
# Preview the Data
orders_df.head()

,ď»żid,ordered_at,delivered_at,order_id
0,0,8/19/2015 22:01,8/19/2015 22:19,767
1,1,11/16/2015 0:12,11/16/2015 0:40,847
2,2,11/27/2015 8:06,11/27/2015 8:08,972
3,3,10/14/2015 1:16,10/14/2015 1:25,390
4,4,11/28/2015 5:10,11/28/2015 5:16,914


In [202]:
order_items_df = pd.read_sql("SELECT * FROM order_items", conn)
group = order_items_df.groupby(['name'])
group.count()

,ď»żid,order_id,amount_paid
name,,,
banana-smoothie,108,108,108
blt,2843,2843,2843
cake,716,716,716
chicken-parm,3470,3470,3470
grilled-cheese,4163,4163,4163
kale-smoothie,72,72,72
orange-juice,997,997,997
soda,2607,2607,2607
tikka-masala,5024,5024,5024


In [203]:
# # invistigate daily order counts
# # first use date function to transform timestamp to date type data

# orders_df = pd.read_sql("select ordered_at from orders order by 1", conn)
# orders_df['ordered_at'] = pd.to_datetime(orders_df['ordered_at']).dt.date

# group = orders_df.groupby(['ordered_at'])
# group.count()
#orders_df = pd.read_sql("SELECT * FROM orders", conn)
orders_df['ordered_at'] = pd.to_datetime(orders_df['ordered_at']).dt.date

group1 = orders_df.groupby(['ordered_at'])
group1.count().head()


                           


,ď»żid,delivered_at,order_id
ordered_at,,,
2015-08-09,1,1,1
2015-08-11,5,5,5
2015-08-12,1,1,1
2015-08-13,2,2,2
2015-08-14,9,9,9


In [213]:
# Merge two dataframes using an inner join
merge_table = pd.merge(orders_df, order_items_df, on="order_id")
merge_table.head()


,ď»żid_x,ordered_at,delivered_at,order_id,ď»żid_y,name,amount_paid
0,0,2015-08-19,8/19/2015 22:19,767,4396,grilled-cheese,9.25
1,0,2015-08-19,8/19/2015 22:19,767,9449,chicken-parm,10.25
2,0,2015-08-19,8/19/2015 22:19,767,11023,orange-juice,5.25
3,0,2015-08-19,8/19/2015 22:19,767,11568,soda,3.75
4,0,2015-08-19,8/19/2015 22:19,767,14979,grilled-cheese,9.25


In [210]:
revenue = merge_table[['ordered_at', 'amount_paid']]
revenue.head(2)

,ordered_at,amount_paid
0,2015-08-19,9.25
1,2015-08-19,10.25


In [214]:
# lookinf for daily revenue
daily_revenue = revenue.groupby(['ordered_at']).sum()
daily_revenue.head()


,amount_paid
ordered_at,
2015-08-09,39.00
2015-08-11,150.25
2015-08-12,24.25
2015-08-13,59.75
2015-08-14,290.00


In [217]:
# invistigate how much revenue generated from kale_smoothie
kale_smoothie = merge_table[['ordered_at','name', 'amount_paid']]
kale_smoothie.head()


,ordered_at,name,amount_paid
0,2015-08-19,grilled-cheese,9.25
1,2015-08-19,chicken-parm,10.25
2,2015-08-19,orange-juice,5.25
3,2015-08-19,soda,3.75
4,2015-08-19,grilled-cheese,9.25


In [238]:
only_kale_revenue = kale_smoothie.loc[kale_smoothie['name'] == 'kale-smoothie', :]
only_kale_daily = only_kale_revenue.groupby(['ordered_at', 'name']).sum()
only_kale_daily.head()

,,amount_paid
ordered_at,name,
2015-08-15,kale-smoothie,8.75
2015-08-17,kale-smoothie,8.75
2015-08-25,kale-smoothie,8.75
2015-08-27,kale-smoothie,8.75
2015-09-01,kale-smoothie,8.75
